In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['svg.fonttype'] = 'none'
# plt.rcParams['font.family'] = 'sans-serif'
# plt.rcParams['font.sans-serif'] = ['Tahoma']
plt.style.use('seaborn-white')
import dateutil
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from scipy import stats
# from schemdraw import flow, Drawing

In [2]:
animals_used = [1091, 1093, 1060, 1062, 1074, 1092, 1102, 1076, 1082, 1101]
diabetic = [1076, 1082, 1101]
impaired = [1060, 1062, 1074, 1092, 1102]
normal = [1091, 1093]

obs = pd.read_excel(r'..\data\metadata\20210421_FBGRBGOGTT_Huishi sample sheet.xlsx', sheet_name='cleaned')
obs['animal'] = obs['animal'].str.strip('TM').astype('int')
obs['datetime'] = obs['when'].apply(dateutil.parser.parse)
animal_phenotypes = pd.read_excel(r'..\data\metadata\20210421_FBGRBGOGTT_Huishi sample sheet.xlsx', sheet_name='animal phenotypes')
animal_phenotypes['animal'] = animal_phenotypes['animal'].str.strip('TM').astype('int')

fg = pd.read_csv(r'..\data\metadata\combined_metab_lipid_file_grouping.csv', index_col=0)
df = pd.read_csv(r'..\data\processed\combined_metab_lipid_table.csv', index_col=0)
data_cols = df.filter(regex='FBG|RBG').columns
pval_cols = df.filter(regex='pval').columns

In [3]:
df

,Type,i,Proposed ID,Tracefinder ID,Lipid Class,m/z,RT [min],Area (Max.),Score,Adduct,...,1092_10_FBG,1082_8_RBG,1062_8_RBG,1102_10_FBG,1093_8_RBG,1074_9_FBG,1062_9_RBG,1101_8_RBG,1092_8_RBG,1092_10_RBG
Unique ID,,,,,,,,,,,,,,,,,,,,,
unknown_141.01586_16.616,metabolite,m_0,unknown,NaN,NaN,141.01586,16.616,31.253941,NaN,[M-H]-1,...,30.682774,30.713015,30.636650,30.699123,30.766149,30.497881,30.660280,30.612707,30.631697,30.794926
Linoleic Acid_279.23234_1.591,metabolite,m_1,Linoleic Acid,NaN,NaN,279.23234,1.591,30.736789,99.2,[M-H]-1,...,30.124329,29.724060,30.109212,30.203473,29.407519,30.130298,29.755386,29.830917,29.260529,29.010637
unknown_496.33981_2.597,metabolite,m_2,unknown,NaN,NaN,496.33981,2.597,30.704673,NaN,[M+H]+1,...,29.577390,29.910002,30.189935,29.744298,29.968890,29.847046,29.784137,29.933457,29.709495,29.593597
trans-Petroselinic acid_281.24793_1.568,metabolite,m_3,trans-Petroselinic acid,NaN,NaN,281.24793,1.568,30.641931,100.0,[M-H]-1,...,30.315600,29.119540,29.492769,29.923218,28.537792,30.129041,29.130426,29.104520,28.779871,28.319689
Uric acid_167.02007_12.732,metabolite,m_4,Uric acid,NaN,NaN,167.02007,12.732,30.533544,88.4,[M-H]-1,...,22.471185,17.976800,26.822243,26.471452,24.519750,24.755634,27.438915,25.684321,17.121551,24.795886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
unknown_897.82532_20.148,lipid,l_1024,unknown,NaN,unknown,897.82532,20.148,21.562814,NaN,NaN,...,18.604841,18.755897,18.996482,20.095162,17.759520,18.908638,18.093365,17.607114,17.326455,16.927718
unknown_974.9118_20.163,lipid,l_1025,unknown,NaN,unknown,974.91180,20.163,22.106402,NaN,NaN,...,16.326408,19.046476,18.653201,18.635659,17.382575,17.575165,18.798950,16.704255,18.420810,17.646532
unknown_593.58716_20.175,lipid,l_1026,unknown,NaN,unknown,593.58716,20.175,21.652712,NaN,NaN,...,18.815858,18.693961,18.778798,20.192346,17.113028,18.925197,17.773412,17.541022,16.974304,16.687194
